In [1]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import copy
import math
import numpy as np
import math
from data_prep import SystemIdentDataset, ControllerDataset, SystemIdentDatasetNormed
import pickle

def evaluate(model, loss_function, val_loader):
    c_error = 0.0
    cos_sim = torch.nn.CosineSimilarity(dim=1)
    running_loss = 0.0
    batch_count = 0
    for _, example in enumerate(tqdm(val_loader), 0):
        inputs,label = example

        with torch.no_grad():
            outputs = model(inputs)
        
        loss = loss_function(outputs, label)
        
        c_error += torch.sum(cos_sim(outputs,label)).item()/val_dloader.batch_size
        running_loss += loss.item()
        batch_count += 1
    
    c_error /= batch_count
    
    return running_loss, c_error

def train(model, num_epochs, loss_function, optimizer, train_loader, val_loader):
    best_loss = []
    best_cerror = []
    val_loss, c_error = evaluate(model, loss_function, val_loader)
    print(f"Initial validation loss: {val_loss}, cosine error: {c_error}")
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}:")
        for _, example in enumerate(tqdm(train_loader), 0):
            inputs,label = example
            
            outputs = model(inputs)

            loss = loss_function(outputs, label)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()
        val_loss, c_error = evaluate(model, loss_function, val_loader)
        print(f"validation loss: {val_loss}, cosine error: {c_error}")

        if epoch < 5:
            best_loss.append((copy.deepcopy(model), val_loss, c_error))
            best_cerror.append((copy.deepcopy(model), val_loss,c_error))
        else:
            for i, entry in enumerate(best_loss):
                m,l,c = entry
                if val_loss < l:
                    best_loss[i] = (copy.deepcopy(model), val_loss, c_error)
                    break
            for i, entry in enumerate(best_cerror):
                m,l,c = entry
                if c_error > c:
                    best_cerror[i] = (copy.deepcopy(model), val_loss, c_error)
                    break
    return best_loss + best_cerror


In [2]:
train_dataset = SystemIdentDatasetNormed(num_examples=1000000)
val_dataset = SystemIdentDatasetNormed(num_examples=100000)
train_dloader = DataLoader(train_dataset, batch_size=256,shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=256, shuffle=True)



In [3]:


# Define the model
model = torch.nn.Sequential(
    torch.nn.Linear(4, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 4, dtype=torch.float64),
)

# Define the loss function
#loss_fn = torch.nn.MSELoss()
csim = torch.nn.CosineSimilarity(dim=1)
loss_fn = lambda x,y: torch.sum(-1*csim(x,y))
# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())
best_models = train(model=model, num_epochs=20, loss_function=loss_fn, optimizer=optimizer, train_loader=train_dloader, val_loader=val_dloader)
for i, model in enumerate(best_models):
    weights, loss, c_error = model
    print(f"Model {i}: loss: {loss}, cosine error: {c_error}")


100%|██████████| 391/391 [00:01<00:00, 208.04it/s]


Initial validation loss: -946.754009970265, cosine error: 0.00945845997812365
Epoch 1:


100%|██████████| 391/391 [00:02<00:00, 184.70it/s]


validation loss: -96698.08016414402, cosine error: 0.966053390386669
Epoch 2:


100%|██████████| 391/391 [00:02<00:00, 192.54it/s]


...

validation loss: -98111.42514920284, cosine error: 0.9801732851382956
Epoch 19:


100%|██████████| 391/391 [00:01<00:00, 198.66it/s]


validation loss: -99005.1650634549, cosine error: 0.9891021126064469
Epoch 20:


100%|██████████| 391/391 [00:01<00:00, 203.32it/s]

validation loss: -98998.02799402179, cosine error: 0.9890308103622701
Model 0: loss: -99588.793778839, cosine error: 0.9949328022981837
Model 1: loss: -99178.0149951576, cosine error: 0.9908289541555867
Model 2: loss: -99005.1650634549, cosine error: 0.9891021126064469
Model 3: loss: -98998.02799402179, cosine error: 0.9890308103622701
Model 4: loss: -98111.42514920284, cosine error: 0.9801732851382956
Model 5: loss: -99588.793778839, cosine error: 0.9949328022981837
Model 6: loss: -99178.0149951576, cosine error: 0.9908289541555867
Model 7: loss: -99005.1650634549, cosine error: 0.9891021126064469
Model 8: loss: -98998.02799402179, cosine error: 0.9890308103622701
Model 9: loss: -98111.42514920284, cosine error: 0.9801732851382956


In [4]:
with open('./emulator_random_inputs3.pkl', 'wb') as f:
    pickle.dump(best_models[0][0], f)

In [36]:
# Now learn to drive emulated plant from state Zo to Zd in K steps where K is a hyperparameter

train_dataset = ControllerDataset(num_examples=1000000)
val_dataset = ControllerDataset(num_examples=100000)
train_dloader = DataLoader(train_dataset, batch_size=256,shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=256, shuffle=True)

class ControllerTrainedEnclosure(torch.nn.Module):
    def __init__(self, emulator_network, K):
        super(self.__class__, self).__init__()
        emulator_network.requires_grad=False 
        self.system_emulator = emulator_network
        self.K = K
        self.network = torch.nn.Sequential(
                torch.nn.Linear(4, 32, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(32, 64, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(64, 64, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(64, 32, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(32, 1, dtype=torch.float64),
                )

    def forward(self, x):
        for _ in range(self.K):
            xnew = torch.zeros_like(x)
            u = self.network(x)
            xnew[:,0] = u.squeeze()
            xnew[:,1:] = x[:,1:]

            dx = self.system_emulator(xnew)
            x = x + dx
            #torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1)
            
        
        return x



In [37]:

with open('./emulator_random_inputs2.pkl', 'rb') as f:
    emulator = pickle.load(f)
model = ControllerTrainedEnclosure(emulator, K=30)

# Define the loss function
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())
best_models = train(model=model, num_epochs=5, loss_function=loss_fn, optimizer=optimizer, train_loader=train_dloader, val_loader=val_dloader)
for i, model in enumerate(best_models):
    weights, loss, c_error = model
    print(f"Model {i}: loss: {loss}, cosine error: {c_error}")

100%|██████████| 79/79 [00:03<00:00, 23.22it/s]


Initial validation loss: 149962.52260807224, cosine error: 0.0004733375169052871
Epoch 1:


100%|██████████| 79/79 [00:04<00:00, 18.15it/s]


validation loss: 359981.7840931482, cosine error: 0.02704468563412509
Epoch 2:


100%|██████████| 79/79 [00:04<00:00, 19.18it/s]


...

validation loss: 358609.25255553244, cosine error: 0.02687181946608139
Epoch 4:


100%|██████████| 79/79 [00:04<00:00, 19.12it/s]


validation loss: 353410.2904779326, cosine error: 0.02665515366050358
Epoch 5:


100%|██████████| 79/79 [00:04<00:00, 18.97it/s]

validation loss: 337965.3851633391, cosine error: 0.026950246391290223
Model 0: loss: 359981.7840931482, cosine error: 0.02704468563412509
Model 1: loss: 358830.5046627023, cosine error: 0.026850415248318663
Model 2: loss: 358609.25255553244, cosine error: 0.02687181946608139
Model 3: loss: 353410.2904779326, cosine error: 0.02665515366050358
Model 4: loss: 337965.3851633391, cosine error: 0.026950246391290223
Model 5: loss: 359981.7840931482, cosine error: 0.02704468563412509
Model 6: loss: 358830.5046627023, cosine error: 0.026850415248318663
Model 7: loss: 358609.25255553244, cosine error: 0.02687181946608139
Model 8: loss: 353410.2904779326, cosine error: 0.02665515366050358
Model 9: loss: 337965.3851633391, cosine error: 0.026950246391290223


In [ ]:
best_models[8][0].network

In [ ]:


a = torch.zeros((1,4),dtype=torch.float64)
#a[0][2] = np.pi + np.pi/20
em = best_models[8][0].system_emulator
nn = best_models[8][0].network

em(a)


In [ ]:
model

In [ ]:

train_dataset = SystemIdentDatasetEuler(num_examples=2000000)
val_dataset = SystemIdentDatasetEuler(num_examples=100000)
train_dloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=256, shuffle=True)

model = torch.nn.Sequential(
    torch.nn.Linear(4, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 4, dtype=torch.float64),
)

# Define the loss function
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())
best_models = train(model=model, num_epochs=25, loss_function=loss_fn, optimizer=optimizer, train_loader=train_dloader, val_loader=val_dloader)
for i, model in enumerate(best_models):
    weights, loss, c_error = model
    print(f"Model {i}: loss: {loss}, cosine error: {c_error}")